In [0]:
use(JTExtension.	DEEP_NETTS_EXTENSION);

In [0]:
var numInputs = 4;
var numOutputs = 3;
Locale.setDefault(Locale.ENGLISH);

## Reading the Dataset

In [0]:
byte[] ZIP_MAGIC_NUMBER = { 0x50, 0x4B, 0x03, 0x04 };
boolean isZipFile(Path path) {
        if (!Files.exists(path) || !Files.isReadable(path) || path.toFile().length() < ZIP_MAGIC_NUMBER.length) {
            return false;
        }
        try (InputStream is = Files.newInputStream(path)) {
            byte[] fileHeader = new byte[ZIP_MAGIC_NUMBER.length];
            if (is.read(fileHeader) != ZIP_MAGIC_NUMBER.length) {
                return false;
            }
            return java.util.Arrays.equals(fileHeader, ZIP_MAGIC_NUMBER);
        } catch (IOException e) {
            System.err.println("Error reading file header for ZIP check.");
            e.printStackTrace();
            return false;
        }
    }

In [0]:
void deleteDirectoryRecursively(Path path) {
        try {
            Files.walk(path)
                 .sorted((a, b) -> b.compareTo(a))
                 .forEach(p -> {
                     try {
                         Files.delete(p);
                     } catch (IOException e) {
                         System.err.println("Failed to delete path: " + p);
                         e.printStackTrace();
                     }
                 });
        } catch (IOException e) {
            System.err.println("Failed to walk directory: " + path);
            e.printStackTrace();
        }
    }

In [0]:
Optional<Path> load(String urlString) {
         Path downloadedFile = null;

        try {
            URL url = new URL(urlString);
            downloadedFile = Files.createTempFile("download-", ".tmp");
            Files.delete(downloadedFile);

            try (InputStream inputStream = url.openStream()) {
                Files.copy(inputStream, downloadedFile);
            }

            if (isZipFile(downloadedFile)) {
                Path tempDir = Files.createTempDirectory("unpacked-");
                try (ZipInputStream zipStream = new ZipInputStream(Files.newInputStream(downloadedFile))) {
                    ZipEntry entry;
                    while ((entry = zipStream.getNextEntry()) != null) {
                        Path entryPath = tempDir.resolve(entry.getName()).normalize();
                        if (!entryPath.startsWith(tempDir)) {
                            // Security check to prevent Zip Slip vulnerability
                            throw new IOException("Zip entry is outside of the target directory: " + entry.getName());
                        }
                        if (entry.isDirectory()) {
                            Files.createDirectories(entryPath);
                        } else {
                            Files.createDirectories(entryPath.getParent());
                            Files.copy(zipStream, entryPath);
                        }
                    }
                }
                Files.delete(downloadedFile);
                return Optional.of(tempDir);
            } else {
               return Optional.of(downloadedFile);
            }

        } catch (IOException e) {
            System.err.println("Failed to download or process file from URL: " + urlString);
            e.printStackTrace();
            if (downloadedFile != null) {
                try {
                    Files.deleteIfExists(downloadedFile);
                } catch (IOException cleanupException) {
                    System.err.println("Failed to clean up downloaded file: " + cleanupException.getMessage());
                }
            }
            return Optional.empty();
        }
    }


In [0]:
var iris = load("https://archive.ics.uci.edu/static/public/53/iris.zip").get().resolve("iris.data");
iris

In [0]:
var df = Csv.loader()
          .floatCol(0)
          .floatCol(1)
          .floatCol(2)
          .floatCol(3)
          .load(iris);
df

In [0]:
import org.dflib.print.TabularPrinter;
import javafx.scene.control.Label;
var printer = new TabularPrinter();

In [0]:
printer.print(df)

In [0]:
import static org.dflib.Exp.*;

In [0]:
var df2 = df.cols(0,1,2,3,4)
    .selectAs("sepal length","sepal width", "petal length", "petal width", "species");
var df3 = df2.colsAppend("Iris Setosa", "Iris Versicolour", "Iris Virginica").merge($col("species").castAsStr().mapVal( s->  

}));

display(new Label(printer.print(df3)));

In [0]:


float[] transform(String s) {
   return switch(s) {
     case "Iris-setosa" -> new float[]{1f,0f,0f};
     case "Iris-versicolor" -> new float[]{0f,1f,0f};
     case "Iris-virginica" -> new float[]{0f,0f,1f};
     default -> throw new IllegalArgumentException("Unknown IrisFlower " + s);
   };
}

var items = StreamSupport
   .stream(Spliterators.spliteratorUnknownSize(df.iterator(), Spliterator.IMMUTABLE), false)
   .map(r -> new TabularDataSet.Item(new float[] {(float)r.get(0),(float) r.get(1), (float) r.get(2),(float) r.get(3)}, transform(r.get(4).toString())))
   .toList();

var dataSet = new TabularDataSet(4,3);
dataSet.setColumnNames("sepal length","sepal width","petal length", "petal width", "Iris Setosa", "Iris Versicolour", "Iris Virginica");
items.forEach(i -> dataSet.add(i));

In [0]:
var scaler = new MaxScaler(dataSet);
scaler.apply(dataSet);        
        
var trainTestSet = dataSet.split(0.6, 0.4);
var trainingSet = trainTestSet[0]; 
var testSet = trainTestSet[1];

var neuralNet = FeedForwardNetwork.builder()
                .addInputLayer(numInputs) 
                .addFullyConnectedLayer(8, ActivationType.RELU) 
                .addOutputLayer(numOutputs, ActivationType.SOFTMAX)
                .lossFunction(LossType.CROSS_ENTROPY) 
                .randomSeed(456)
                .build();

record EpochAndLoss(int epoch, float loss) {}

var trainingEvents = new ArrayList<EpochAndLoss>();
var trainer = neuralNet.getTrainer();
trainer.setMaxError(0.04f);
trainer.setLearningRate(0.01f);
trainer.setOptimizer(OptimizerType.MOMENTUM);
trainer.setMomentum(0.9f); 
trainer.addListener(e -> trainingEvents.add(
  new EpochAndLoss(e.getSource().getCurrentEpoch(), e.getSource().getTrainingLoss())
));
                
neuralNet.train(trainingSet);
         
var evaluator = new ClassifierEvaluator();
var em = evaluator.evaluate(neuralNet, testSet);
println("CLASSIFIER EVALUATION METRICS"); 
em

## Confusion Matrix

In [0]:
evaluator.getConfusionMatrix()

In [0]:
dataSet.getColumnNames()

In [0]:
dataSet.getTargetColumnsNames()

In [0]:
enum Column { SEPAL_LENGTH, SEPAL_WIDTH, PETAL_LENGTH, PETAL_WIDTH }

In [0]:
enum IrisFlowerClass { SETOSA, VERSICOLOR, VIRGINICA }

In [0]:
record Row (float sepal_length, float sepal_width, float petal_length, float petal_width, IrisFlowerClass irisFlower) {}

In [0]:
var r = (TabularDataSet.Item) dataSet.getItems().stream().findFirst().get();
r.getTargetOutput()

In [0]:
var x = trainingEvents.stream().mapToInt(e -> e.epoch()).toArray();
var y = trainingEvents.stream().mapToDouble(e -> e.loss()).toArray();
x.length

In [0]:
var chart = lineFx(x,y, "Epoch", "Training Loss", "Training Run");

In [0]:
display(chart)

In [0]:
df.getColumnsIndex()